In [2]:
project_directory = "/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13"
import sys
sys.path.append(project_directory)

In [3]:
import carla
import yaml
import os

from utils_cogmod.server_utils import CarlaServerManager

In [4]:
# file_path = os.path.join(project_directory, 'outputs', '2024-04-26', '14-41-15')
# diagnostics = os.path.join(file_path, 'diagnostics')

In [5]:
def read_config(config_path):
    with open(config_path, 'r') as file:
        cfg = yaml.safe_load(file)
    return cfg

def init_client(host, port):
    try:
        client = carla.Client(host, port)
        print(f"client connected to {host}:{port}")
        print(f"client.get_server_version(): {client.get_server_version()}, client.get_client_version(): {client.get_client_version()}")
        client.set_timeout(60.0)
    except RuntimeError as re:
        if "timeout" not in str(re) and "time-out" not in str(re):
            print("Could not connect to Carla server because:", re)
        client = None 
    return client

# Assuming 'client' is already initialized and is an instance of carla.Client
# The path to your recorder log file

def run_replay(config_path, recorder_file_path, actor_id, start_time, duration):
    cfg = read_config(config_path)
    # server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
    # server_manager.stop()
    # server_manager.start()

    client = init_client(cfg['host'], cfg['port'])
    # Check if file exists
    try:
        with open(recorder_file_path, 'r') as file:
            print(f"File found: {recorder_file_path}")
    except FileNotFoundError:
        print(f"File not found: {recorder_file_path}")
        exit()
    try:
        # Corrected call to replay_file
        replay_result = client.replay_file(
            recorder_file_path,
            time_start=start_time,  # Explicitly specifying as double
            duration=duration,    # Explicitly specifying as double
            follow_id=actor_id,     # This is already an int, but making sure it's clearly specified
            replay_sensors=True  # Assuming you want to replay sensors, adjust as necessary
        )
        print("Replaying recorded log file...", replay_result)
    except Exception as e:
        print(f"An error occurred while trying to replay the file: {e}")

In [6]:
config_path = r"/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/config/benchmark.yaml"
recorder_file_path = r"/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log"

In [7]:
cfg = read_config(config_path)
server_manager = CarlaServerManager(cfg['carla_sh_path'], cfg['port'], t_sleep=5)
server_manager.stop()
server_manager.start()

CarlaUE4-Linux: no process found


In [8]:
client = init_client(cfg['host'], cfg['port'])

all_info = client.show_recorder_file_info(recorder_file_path, True)

client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13


In [9]:
# # save all info in a text file 
# with open(os.path.join(r"C:\Users\abjawad\Documents\GitHub\carla-roach-0.9.13\notebooks", 'all_info.txt'), 'w') as f:
#     f.write(all_info)

In [10]:
collision_info = client.show_recorder_collisions(recorder_file_path, "h", "v")

print(type(collision_info))
print(collision_info)

<class 'str'>
Version: 1
Map: Town04
Date: 04/28/24 01:42:28

    Time  Types     Id Actor 1                                 Id Actor 2                            
     174   v v    4763 vehicle.lincoln.mkz_2017              4929 vehicle.tesla.cybertruck           

Frames: 2375
Duration: 237 seconds



In [11]:
#Gets gaze direction and other data at time of the crash
import csv
import json

collision_info = client.show_recorder_collisions(recorder_file_path, "h", "v")

lines = collision_info.splitlines()

filename = recorder_file_path[: len(recorder_file_path) - 3] + 'csv'
filename2 = recorder_file_path[: len(recorder_file_path) - 3] + 'json'

f = open(filename2)
data = json.load(f)

collision_vehicles = data['hero']['collisions_vehicle']
collision_pedestrian = data['hero']['collisions_pedestrian']

valid_gazes = 0

if (len(lines) > 7):
   for i in range(len(lines) - 8):
       error_count = 0

       j = i + 5
       line = lines[j].strip()
       time = int(line.split()[0])

       if (i == 0):
           time_prior = 0
       else:
           line_prior = lines[j - 1].strip()
           time_prior = int(line_prior.split()[0])

       if (time_prior >= time - 3 and time_prior <= time + 3):
           error_count += 1

       row_num = time + 1
       row_num_prior = time_prior + 1
       counter = 0
       counter_prior = 0

       with open(filename, mode = 'r') as file:
           csv_reader = csv.reader(file)
           header = next(csv_reader)

           for row in csv_reader:
               counter_prior += 1
               if counter_prior == row_num_prior:
                   prior_gaze = row[13]
                  
       with open(filename, mode = 'r') as file:
           csv_reader = csv.reader(file)
           header = next(csv_reader)

           for row in csv_reader:
               counter += 1
               if counter == row_num:
                   gaze = row[13]

                   if prior_gaze == gaze:
                       error_count += 1
                   break

       if error_count == 2:
           continue
       else:
           print(f"Information about crash {valid_gazes + 1} in this experiment on the Carla simulator:")
          
           print()

           print("Gaze direction at time of crash: {}".format(row[13]))

           intensity = collision_vehicles[valid_gazes].get('intensity')
           print("Intensity: {}".format(intensity))

           #print("Time: {}".format(time))
      
           #step = collision_vehicles[valid_gazes].get('step')
           #print("Step: {}".format(step))

           simulation_time = collision_vehicles[valid_gazes].get('simulation_time')
           print("Simulation Time: {}".format(simulation_time))

           collision_type = collision_vehicles[valid_gazes].get('collision_type')
           print("Collision Type: {}".format(collision_type))

           other_actor_id = collision_vehicles[valid_gazes].get('other_actor_id')
           print("Other Actor ID: {}".format(other_actor_id))

           other_actor_type_id = collision_vehicles[valid_gazes].get('other_actor_type_id')
           print("Other Actor Type ID: {}".format(other_actor_type_id))

           print()

           valid_gazes += 1

Information about crash 1 in this experiment on the Carla simulator:

Gaze direction at time of crash: GazeDirection.LEFTBLINDSPOT
Intensity: 3197.226972899177
Simulation Time: 172.70000257343054
Collision Type: 1
Other Actor ID: 4929
Other Actor Type ID: vehicle.tesla.cybertruck



In [ ]:
import csv
import pandas as pd

csv_path = r"/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple"
csv_name = "distracted_1_repeat_1_replay.csv"

full_csv_path = os.path.join(csv_path, csv_name)

start = time - 2
end = time + 2

first_time = True
first_time_distance = 0

oldh_x = 0
oldh_y = 0
oldh_z = 0
oldo_x = 0
oldo_y = 0
oldo_z = 0

velocityh = 0
velocityo = 0
oldvelocityh = 0
oldvelocityo = 0

accelerationh = 0
accelerationo = 0

world = client.get_world()

actor_id = 4763

while start < end:
    start = start + 0.1
    client.replay_file(
        recorder_file_path,
        time_start = start,  # Explicitly specifying as double
        duration = 0.01,    # Explicitly specifying as double
        follow_id = actor_id,     # This is already an int, but making sure it's clearly specified
        replay_sensors = True  # Assuming you want to replay sensors, adjust as necessary
    )
 
    hero = world.get_actor(actor_id)
    other_vehicle = world.get_actor(other_actor_id)

    print(start)

    print(f"Velocity hero = {velocityh}")
    print(f"Velocity other = {velocityo}")

    print(f"Acceleration hero = {accelerationh}")
    print(f"Acceleration other = {accelerationo}")

    with open(full_csv_path, 'a', newline = '') as file:
        writer = csv.writer(file)

        if first_time_distance == 0:
            first_time_distance += 1

            oldh_x = hero.get_location().x
            oldh_y = hero.get_location().y
            oldh_z = hero.get_location().z

            oldo_x = other_vehicle.get_location().x
            oldo_y = other_vehicle.get_location().y
            oldo_z = other_vehicle.get_location().z

            continue
        else:
            oldvelocityh = velocityh
            oldvelocityo = velocityo

            velocityh = ((hero.get_location().x - oldh_x) ** 2 + (hero.get_location().y - oldh_y) ** 2 + (hero.get_location().z - oldh_z) ** 2) ** 0.5/0.1 * 2.23694
            velocityo = ((other_vehicle.get_location().x - oldo_x) ** 2 + (other_vehicle.get_location().y - oldo_y) ** 2 + (other_vehicle.get_location().z - oldo_z) ** 2) ** 0.5/0.1 * 2.23694
           
            accelerationh = (velocityh * 0.44704 - oldvelocityh * 0.44704)/0.1
            accelerationo = (velocityo * 0.44704 - oldvelocityo * 0.44704)/0.1

            oldh_x = hero.get_location().x
            oldh_y = hero.get_location().y
            oldh_z = hero.get_location().z

            oldo_x = other_vehicle.get_location().x
            oldo_y = other_vehicle.get_location().y
            oldo_z = other_vehicle.get_location().z

        if first_time == True:
            field = ['simulation_time',
            'ego_vehicle_x_coordinate',
            'ego_vehicle_y_coordinate',
            'ego_vehicle_z_coordinate',
            'ego_vehicle_velocity',
            'ego_vehicle_acceleration',
            'other_vehicle_x_coordinate',
            'other_vehicle_x_coordinate',
            'other_vehicle_x_coordinate',
            'other_vehicle_velocity',
            'other_vehicle_acceleration',
            ]
 
            writer.writerow(field)

            first_time = False

        writer.writerow(
            [start,
            hero.get_location().x,
            hero.get_location().y,
            hero.get_location().z,
            velocityh,
            accelerationh,
            other_vehicle.get_location().x,
            other_vehicle.get_location().y,
            other_vehicle.get_location().z,
            velocityo,
            accelerationo
            ])
       
df = pd.read_csv(full_csv_path)

df.at[0, 'ego_vehicle_velocity'] = 0
df.at[0, 'ego_vehicle_acceleration'] = 0
df.at[1, 'ego_vehicle_acceleration'] = 0

df.at[0, 'other_vehicle_velocity'] = 0
df.at[0, 'other_vehicle_acceleration'] = 0
df.at[1, 'other_vehicle_acceleration'] = 0

df.to_csv(full_csv_path, index = False)

In [15]:
import numpy as np
import pygame

start_time = time - 3
duration = time + 3
actor_id = 4763
frame_count = 0

client = init_client(cfg['host'], cfg['port'])

run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

# Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
world = client.get_world()

settings = world.get_settings()
settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)

blueprint_library = world.get_blueprint_library()
actor = world.get_actor(actor_id)

# Add a camera sensor to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=actor)

def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (image.height, image.width, 4))
    array = array[:, :, :3]  # Remove alpha channel
    return array

def save_image(image, output_path):
    image.save_to_disk(output_path)

def image_callback(image):
    global frame_count
    output_path = os.path.join(output_directory, f"image_{frame_count:06d}.png")
    save_image(image, output_path)
    frame_count += 1

camera.listen(image_callback)

output_directory = "/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_ego_c1"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

snapshot = world.get_snapshot()
start_simulation_time = snapshot.timestamp.elapsed_seconds
end_simulation_time = start_simulation_time + 10

while True:
    snapshot = world.get_snapshot()
    current_time = snapshot.timestamp.elapsed_seconds

    if current_time < end_simulation_time:
        world.tick()
    else:
        break
    
camera.stop()
camera.destroy()
actor.destroy()

client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13
client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13
File found: /home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log
Replaying recorded log file... Replaying File: /home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log
Total time recorded: 237.4
Replaying from 171 s - 348 s (237.4 s) at 1.0x



True

In [16]:
import numpy as np
import pygame

start_time = time - 3
duration = time + 3
actor_id = 4763
frame_count = 0

client = init_client(cfg['host'], cfg['port'])

run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

# Get the world and set no rendering mode and create blueprint library to access rgb camera sensor
world = client.get_world()

settings = world.get_settings()
settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)

blueprint_library = world.get_blueprint_library()
other_actor = world.get_actor(other_actor_id)

# Add a camera sensor to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=other_actor)

def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
    array = np.reshape(array, (image.height, image.width, 4))
    array = array[:, :, :3]  # Remove alpha channel
    return array

def save_image(image, output_path):
    image.save_to_disk(output_path)

def image_callback(image):
    global frame_count
    output_path = os.path.join(output_directory, f"image_{frame_count:06d}.png")
    save_image(image, output_path)
    frame_count += 1

camera.listen(image_callback)

output_directory = "/home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/crash_images/experiment_1/00-06-31/CogMod-v0_Town04_high_simple/distracted_1_repeat_1_other_c1"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

snapshot = world.get_snapshot()
start_simulation_time = snapshot.timestamp.elapsed_seconds
end_simulation_time = start_simulation_time + 10

while True:
    snapshot = world.get_snapshot()
    current_time = snapshot.timestamp.elapsed_seconds

    if current_time < end_simulation_time:
        world.tick()
    else:
        break
    
camera.stop()
camera.destroy()
other_actor.destroy()

client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13
client connected to localhost:2000
client.get_server_version(): 0.9.13, client.get_client_version(): 0.9.13
File found: /home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log
Replaying recorded log file... Replaying File: /home/ubuntu/vihaan-devel/carla/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_1.log
Total time recorded: 237.4
Replaying from 171 s - 348 s (237.4 s) at 1.0x



RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [ ]:
start_time = 130
duration = 0
actor_id = 227
other_actor_id = 287

run_replay(config_path, recorder_file_path, actor_id, start_time, duration)

In [ ]:
world = client.get_world()
actor = world.get_actor(actor_id)
actor.destroy()

In [ ]:
path = "/home/abjawad/carla-sip/carla-roach-0.9.13/outputs/experiment_1/00-06-31/diagnostics/CogMod-v0_Town04_high_simple/distracted_1_repeat_2.log"
counter = 0


with open(path, 'r', encoding = 'utf-8', errors = 'ignore') as file:
   log_data = file.readlines()


   print(len(log_data))
   print(type(log_data[0]))


   for line in log_data:
       if 'hero' in line:
           #print(line[0: 100])
           print(line)


   '''
   for i in range(len(log_data)):
       if 'frame' in log_data[counter]:
           print(line[0: 100])
       counter += 1
   '''